In [ ]:
import sys
if "google.colab" in str(get_ipython()):
    ! {sys.executable} -m pip install pytorch-lifestream

In [ ]:
import os
import torch
import pandas as pd
import pytorch_lightning as pl
import numpy as np
from datetime import timedelta
from torch import tensor

from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames import PtlsDataModule
from ptls.frames.coles import CoLESModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.datasets import inference_data_loader
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.preprocessing import PandasDataPreprocessor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import pickle
from matplotlib import pyplot as plt
from ptls.frames.coles.losses import ContrastiveLoss
from ptls.frames.coles.sampling_strategies import HardNegativePairSelector
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_data = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true', compression='gzip')
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [ ]:
path = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true"
df_target = pd.read_csv(path)
df_target = df_target.set_index("client_id")
df_target.rename(columns={"bins": "target"}, inplace=True)
df_target.head()

,target
client_id,
24662,2
1046,0
34089,2
34848,1
47076,3


In [ ]:
id_col = 'client_id'
date_col = 'trans_date'

NUM_EXP = 102

numerical = ['amount_rur']
categorical = ['small_group', 'trans_date']

emb_values = {'small_group': 202, 'trans_date': 730}

In [ ]:
import random

def generate_random_features(numerical, categorical):
    # Генерация случайного выбора для числовых фич
    num_sample = random.choice([None, random.sample(numerical, random.randint(1, len(numerical)))])

    # Генерация случайного выбора для категориальных фич
    cat_sample = random.choice([None, random.sample(categorical, random.randint(1, len(categorical)))])

    # Если не выбрана ни одна фича, возвращаем None для обоих категорий
    if num_sample is None and cat_sample is None:
        return {'numerical': None, 'categorical': None}

    return {'numerical': num_sample, 'categorical': cat_sample}

In [ ]:
features =  generate_random_features(numerical, categorical)
num, cat = features['numerical'], features['categorical']
embeds = {}
if cat is not None:
  for j in cat:
    random_dim = np.random.choice([32, 16, 8, 4])
    # parameters[i][f'emb_dim_{j}'] = random_dim
    embeds.update({j: {"in": emb_values[j], "out": random_dim}})

In [ ]:
num_size = len(num) if num!= None else 0
emb_size = sum([val['out'] for val in embeds.values()])

In [ ]:
import pickle
import json

def save_res(dct, path):
    # Сохранение в бинарном формате с использованием pickle
    with open(path + '.pkl', 'wb') as handle:
        pickle.dump(dct, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # # Сохранение в читаемом формате (JSON) в .txt
    # with open(path + '.txt', 'w', encoding='utf-8') as handle:
    #     json.dump(dct, handle, indent=4, ensure_ascii=False)

    # Сохранение в читаемом формате (Python-код) в .py
    with open(path + '.py', 'w', encoding='utf-8') as handle:
        handle.write(f"data = {repr(dct)}")

In [ ]:
# parameters
PATH = '/content/drive/MyDrive/PTLS/data_ensemble_age_group/train_params'
with open(PATH + '.pkl', 'rb') as handle:
   parameters = pickle.load(handle)

In [ ]:
for i in range(NUM_EXP):
    while True:

      try:

        parameters[i] = {}

        all_features = [id_col, date_col]
        num, cat = None, None
        while num== None and cat==None:
          features =  generate_random_features(numerical, categorical)
          num, cat = features['numerical'], features['categorical']


        parameters[i]['num'], parameters[i]['cat'] = num, cat

        if cat is not None:
          all_features +=cat
        if num is not None:
          all_features +=num
        all_features = list(set(all_features))

        data = source_data[all_features]

        source_preprocessor = PandasDataPreprocessor(
          col_id=id_col,
          col_event_time=date_col,
          event_time_transformation="none",
          cols_numerical=num,
          cols_category = cat,
          return_records=True,
        )

        dataset = source_preprocessor.fit_transform(data)
        dataset = sorted(dataset, key=lambda x: x[id_col])


        train, test = train_test_split(dataset, test_size=0.2, random_state=42)

        # все кат фичи переводим в размерность рандомную из списка

        embeds = {}
        if cat is not None:
          for j in cat:
            random_dim = np.random.choice([32, 16, 8, 4])
            parameters[i][f'emb_dim_{j}'] = random_dim
            embeds.update({j: {"in": emb_values[j], "out": random_dim}})


        num_size = len(num) if num!= None else 0
        emb_size = sum([val['out'] for val in embeds.values()])
        rnn_inp_size = num_size + emb_size

        random_noise = np.random.uniform(0, 0.1)
        parameters[i]['random_noise'] = random_noise
        random_linear_projection = np.random.choice([0, 32, 64, 128, 256])
        parameters[i]['random_linear_projection'] = random_linear_projection


        trx_encoder_params = dict(
            embeddings_noise = random_noise,
            linear_projection_size = random_linear_projection,
            embeddings = embeds,
            numeric_values = {i:'identity' for i in num} if num is not None else None,
        )


        random_type = np.random.choice(['lstm', 'gru'])
        parameters[i]['random_type'] = random_type


        seq_encoder = RnnSeqEncoder(
            trx_encoder=TrxEncoder(**trx_encoder_params),
            input_size = random_linear_projection if random_linear_projection is not None else rnn_inp_size,
            hidden_size=256,
            type=random_type,
        )



        random_lr = float(np.random.choice([0.001, 0.005, 0.01, 0.0003, 0.0001]))
        parameters[i]['random_lr'] = random_lr
        random_margin = float(np.random.choice(np.arange(0.1, 1.1, 0.1)))
        parameters[i]['random_margin'] = random_margin
        random_neg_count = int(np.random.choice(np.arange(1, 4, 1)))
        parameters[i]['random_neg_count'] = random_neg_count


        loss = ContrastiveLoss(margin=random_margin, sampling_strategy=HardNegativePairSelector(neg_count=random_neg_count))
        model = CoLESModule(
            seq_encoder=seq_encoder,
            optimizer_partial=partial(torch.optim.Adam, lr = random_lr),
            lr_scheduler_partial=partial(
                torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9
            ),
        )

        random_split_count = np.random.choice(np.arange(1, 5, 1))
        parameters[i]['random_split_count'] = random_split_count
        random_cnt_min = np.random.choice(np.arange(10, 50, 5))
        parameters[i]['random_cnt_min'] = random_cnt_min
        random_cnt_max = np.random.choice(np.arange(random_cnt_min+5, 250, 5))
        parameters[i]['random_cnt_max'] = random_cnt_max


        train_dl = PtlsDataModule(
            train_data=ColesDataset(
                MemoryMapDataset(
                    data=train,
                    i_filters=[SeqLenFilter(min_seq_len=25)],
                ),
                splitter=SampleSlices(
                    split_count=random_split_count,
                    cnt_min=random_cnt_min,
                    cnt_max=random_cnt_max,
                ),
            ),
            train_num_workers=16,
            train_batch_size=256,
        )


        random_max_epochs = int(np.random.choice(np.arange(5, 18, 1)))
        parameters[i]['random_max_epochs'] = random_max_epochs

        trainer = pl.Trainer(
            max_epochs=random_max_epochs,
            accelerator="cuda" if torch.cuda.is_available() else "cpu",
            enable_progress_bar=False,
        )

        trainer.fit(model, train_dl)
        print(trainer.logged_metrics)
        parameters[i]['logged_metrics'] = trainer.logged_metrics

        train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
        train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

        test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
        test_embeds = torch.vstack(trainer.predict(model, test_dl))


        train_df = pd.DataFrame(data=train_embeds, columns=[f"embed_{i}" for i in range(train_embeds.shape[1])])
        train_df["client_id"] = [x["client_id"] for x in train]
        train_df = train_df.merge(df_target, how="left", on="client_id")

        test_df = pd.DataFrame(data=test_embeds, columns=[f"embed_{i}" for i in range(test_embeds.shape[1])])
        test_df["client_id"] = [x["client_id"] for x in test]
        test_df = test_df.merge(df_target, how="left", on="client_id")


        train_df.to_csv(f'/content/drive/MyDrive/PTLS/data_ensemble_age_group/train_df{i}.csv')
        test_df.to_csv(f'/content/drive/MyDrive/PTLS/data_ensemble_age_group/test_df{i}.csv')
        save_res(parameters, PATH)
        print(parameters[i])
        break

      except Exception as e:
        print(f"Ошибка на итерации {i}: {e}")
        continue  # Повторяем итерацию с тем же i

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 247 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
247 K     Trainable params
0

{'loss': tensor(260.4991), 'seq_len': tensor(100.8411)}


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'num': ['amount_rur'], 'cat': ['trans_date'], 'emb_dim_trans_date': 32, 'random_noise': 0.07227087102608425, 'random_linear_projection': 32, 'random_type': 'gru', 'random_lr': 0.005, 'random_margin': 0.30000000000000004, 'random_neg_count': 3, 'random_split_count': 4, 'random_cnt_min': 40, 'random_cnt_max': 160, 'random_max_epochs': 12, 'logged_metrics': {'loss': tensor(260.4991), 'seq_len': tensor(100.8411)}}


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 223 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
223 K     Trainable params
0

...

{'num': None, 'cat': ['trans_date', 'small_group'], 'emb_dim_trans_date': 32, 'emb_dim_small_group': 16, 'random_noise': 0.07661867016717669, 'random_linear_projection': 256, 'random_type': 'gru', 'random_lr': 0.001, 'random_margin': 0.2, 'random_neg_count': 1, 'random_split_count': 1, 'random_cnt_min': 15, 'random_cnt_max': 75, 'random_max_epochs': 13, 'logged_metrics': {'loss': tensor(0.), 'seq_len': tensor(46.5938)}}


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 303 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
303 K     Trainable params
0

{'loss': tensor(274.2792), 'seq_len': tensor(20.0430)}


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'num': ['amount_rur'], 'cat': ['trans_date'], 'emb_dim_trans_date': 8, 'random_noise': 0.026166513280216852, 'random_linear_projection': 32, 'random_type': 'lstm', 'random_lr': 0.01, 'random_margin': 0.30000000000000004, 'random_neg_count': 1, 'random_split_count': 4, 'random_cnt_min': 15, 'random_cnt_max': 25, 'random_max_epochs': 9, 'logged_metrics': {'loss': tensor(274.2792), 'seq_len': tensor(20.0430)}}


In [ ]:
from google.colab import runtime
runtime.unassign()